# Pandas | Core concepts, types and methods

Szymon Talaga | 08.01.2020

<hr>

In this notebook we will carefully develop proper and in-depth understanding of core concepts, types (classes) and methods provided by Pandas.
They will be discussed with respect to Numpy in order to facilitate understanding of what typical usecases of both packages are as well as what
are their strong and weak points.

Pandas is focused around two main data structures:

* `Series` : a one-dimensional sequence of values with a fixed data type.
* `DataFrame` : a two-dimensional rectangular table of values with rows and columns. Each column has to be of fixed data type as it is represented as `Series` objects.

In fact, Pandas provides more data structures, but they are useful in rather specific circumstances so we will not discuss them here.

<hr>

Pandas is internally based on Numpy and its types such as `Series` and `DataFrame` are to a large extent compatible with many Numpy functions.
However, there are also very important differences between them.

The crucial difference concerns the way axes and their indexes are designed which also means that the notion of broadcasting in Pandas is completely different.

In Numpy indexes of axes are defined **implicitly** by the fact that elements are arranged sequentially, so they can be assigned with integer coordinates
ranging from $0$ to $n-1$ where $n$ is the number elements along a given axis.

In Pandas the way indexes work is entirely different. Here indexes are **explicit**, they exist as separate Python objects, and are defined as sets of labels (or even multilevel hierarchies of labels)
for elements along a given axis. Therefore, the rules of broadcasting in Pandas are not determined by conformability of shapes of arrays like in Numpy, but by alignment (congruence) of labels.
We will discuss the details of broadcasting and alignment in Pandas while discussing `Series` and `DataFrame` types.

<hr>

Many great resources about Pandas can be found in the official documentation. In particular, it is recommended to read the following articles:

* [10 minutes to Pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html)
* [Essential basic functionality](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html)
* [Intro to data structures](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html)
* [Indexing and selecting data](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-attribute-access)

<hr>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
### Configure IPython shell to show print all outputs generated in a code cell
### --------------------------------------------------------------------------
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Series | 1D sequences of values with fixed dtypes

The simpler and more basic of the two core types in Pandas is `Series`. Objects of this type are used to store one-dimensional ordered sequences of values of fixed data type
(in most cases these are standard Numpy `dtypes`) aligned along an index which is a sequence of labels that identify individual elements. So a series has the following structure:

```python
Index  |  Value
===============
  b    |    5
  a    |    3
  f    |    1
  c    |    3
===============
```

Importantly, `Series` objects are both like dictionaries / mappings as well as lists / 1D arrays. In other words, they support both label / key based indexing characteristic for mappings 
and numerical / positional indexing characteristics for sequences / lists.

A `Series` object may be created with a type constructor from any object that can be interpreted as 1D array / sequence.

In [5]:
## Sequence-based initialization
s = pd.Series([5, 3, 1, 3])
s

0    5
1    3
2    1
3    3
dtype: int64

The first column in the print output shows indexes and the second data values. Note, that in this case a default generic index was created (sequence from `0` to `n-1`).
However, if we want we may explicitly assign an index composed of any values we want.

In [6]:
## Sequence-based initialization with arbitrary index
s = pd.Series([5, 3, 1, 3], index=['b', 'a', 'f', 'c'])
s

b    5
a    3
f    1
c    3
dtype: int64

Another very convenient approach is to create a series from dictionary.

In [7]:
## Dictionary-based initialization
s = pd.Series({ 'b': 5, 'a': 3, 'f': 1, 'c': 3 })
s

b    5
a    3
f    1
c    3
dtype: int64

Look good. But what about order? We discussed the fact that dictionaries in Python are inherently unordered. That stops to be true recently.
From the version 3.7 of Python `dict` objects keeps the insertion order (the order in which their keys were provided).
Pandas uses this fact when creating `Series` objects. In older version of Python and/or Pandas index is sorted lexicographically when
a series is built from a dictionary.

If a series is initialized with values of different types it will be upcasted to the most general data type.

In [8]:
# Upcasting of integers to floats
pd.Series([1, 2, 3.5])

0    1.0
1    2.0
2    3.5
dtype: float64

In [10]:
# Upcasting to arbitrary python objects
pd.Series([1, 'string', [1,2,3]])

0            1
1       string
2    [1, 2, 3]
dtype: object

A series can be also created from a scalar (single) value. If additionally an index is provided then a series as long as the index is created
and it is populated with the constant value.

In [11]:
## Single value series as there is no index
pd.Series(5)

0    5
dtype: int64

In [12]:
## Series from scalar and index
pd.Series(5, index=['a', 'b', 'c'])

a    5
b    5
c    5
dtype: int64

Index of a series can be retrieved via the `.index` attribute

In [13]:
s.index

Index(['b', 'a', 'f', 'c'], dtype='object')

In [14]:
s.values

array([5, 3, 1, 3])

Numpy representation of the underlying data can be retrieved with `.to_numpy()` method.

In [15]:
s

b    5
a    3
f    1
c    3
dtype: int64

In [16]:
s.to_numpy()

array([5, 3, 1, 3])

Moreover, series can be assigned with names. We will see how it can be useful later on when we discuss data frames.

In [18]:
s.name = 'a series'
s

b    5
a    3
f    1
c    3
Name: a series, dtype: int64

In [19]:
pd.Series([1, 2, 3], name='name')

0    1
1    2
2    3
Name: name, dtype: int64

### Series | Indexing & slicing

Similarily to data frames series support three main types of indexing.

#### Standard dictionary syntax (aka _getitem_)

It is convenient as it allows to use both label-based and position-based indexing and slicing. However, due to its flexibility it can be
confusing as it has to guess what is the intent of the user based on the type of indexer.

That is why it is usually better to use more explicit `.loc` and `.iloc` indexers.

In [20]:
s

b    5
a    3
f    1
c    3
Name: a series, dtype: int64

In [21]:
# Label-based index
s['f']

1

In [23]:
L = [0, 1, 2, 3, 4]
L[1:3]

[1, 2]

In [24]:
# Label-based slice
s['a':'c']

a    3
f    1
c    3
Name: a series, dtype: int64

In [25]:
# List of labels
s[['a', 'f']]

a    3
f    1
Name: a series, dtype: int64

In [26]:
# Position-based (integer) index
s[2]

1

In [28]:
# Position based (integer) slice
s
s[1:3]

b    5
a    3
f    1
c    3
Name: a series, dtype: int64

a    3
f    1
Name: a series, dtype: int64

In [29]:
# List of (integer) positions
s[[0, 2]]

b    5
f    1
Name: a series, dtype: int64

In [30]:
x = pd.Series([2, 3, 4, 5], index=[2, 0, 3, 1])
x

2    2
0    3
3    4
1    5
dtype: int64

In [35]:
x[0]
x[0:2]

3

2    2
0    3
dtype: int64

This is all nice. However, this is possible because integer indexers are also interpreted as positional indexers.
This means that with this method we **can not** perform label indexing for series with integer labels.

#### Label-based indexing (aka `.loc` indexing)

The `.loc` attribute is defined on every `Series` (as well as `DataFrame`) object and it returns the special indexer object that allows
to query our data based on the explicit index labels, even if they are integers.

NOTE. Slices in `.loc` indexer always return ranges **including** the rightmost element.

In [36]:
s

b    5
a    3
f    1
c    3
Name: a series, dtype: int64

In [38]:
s.loc

In [39]:
s.loc['a']

3

In [40]:
s.loc['a':'f']

a    3
f    1
Name: a series, dtype: int64

Label `.loc` indexing can also use lists of labels.

In [2]:
s.loc[['a', 'f']]

NameError: name 's' is not defined

In [41]:
# Works with integers labels too
x = pd.Series([1, 2, 3], index=[2, 0, 1])
x

2    1
0    2
1    3
dtype: int64

In [1]:
x.loc[0]

NameError: name 'x' is not defined

In [44]:
x.loc[0:1]

0    2
1    3
dtype: int64

#### Positional indexing (aka `.iloc` indexing)

The `.iloc` attribute returns a specialized indexer object that allows to query our data with positional (integer) indexes.

NOTE. Slices in `.iloc` follows standard Python semantics and **do not** include the rightmost element.

In [46]:
x

2    1
0    2
1    3
dtype: int64

In [47]:
x.iloc[0]

1

In [48]:
x.iloc[0:2]

2    1
0    2
dtype: int64

Positional `.iloc` indexing can be also used like integer indexing in Numpy.

In [49]:
s.iloc[[0, 2]]

b    5
f    1
Name: a series, dtype: int64

#### Boolean indexing

There is also the fourth type of indexing that can be performed equally well with _getitem_, `.loc` and `.iloc` indexers.
It is of course boolean indexing, which works the same as in Numpy.

In [51]:
mask = [True, False, False, True]

# getitem
s[mask]

In [51]:
# .loc
s.loc[mask]

In [51]:
# .iloc
s.iloc[mask]

b    5
c    3
Name: a series, dtype: int64
b    5
c    3
Name: a series, dtype: int64
b    5
c    3
Name: a series, dtype: int64


## Series | setting values

All types of indexing discussed above can be used to set new values to a series, including creating new entries with new labels.

In [53]:
x = pd.Series({ 'Alice': 10, 'Bob': 11 })
print("\n", x)


 Alice    10
Bob      11
dtype: int64


In [54]:
x['Alice'] = 9
print("\n", x)


 Alice     9
Bob      11
dtype: int64


In [55]:
x.loc['Bob'] = 0
print("\n", x)


 Alice    9
Bob      0
dtype: int64


In [56]:
x.iloc[0] = 100
print("\n", x)


 Alice    100
Bob        0
dtype: int64


In [57]:
x.loc[['Alice', 'Bob']] = 1
print("\n", x)


 Alice    1
Bob      1
dtype: int64


In [59]:
x.loc['Mark'] = 777
x

Alice      1
Bob        1
Mark     777
dtype: int64

### Series | vectorization between series and scalars

Vectorization between series and scalars in Pandas is as simple and trivial as in Numpy.

In [61]:
s

b    5
a    3
f    1
c    3
Name: a series, dtype: int64


In [62]:
## Addition
s + 2

b    7
a    5
f    3
c    5
Name: a series, dtype: int64


In [63]:
## Subtraction
s - 2

b    3
a    1
f   -1
c    1
Name: a series, dtype: int64


In [64]:
## Multiplication
s * 2

b    10
a     6
f     2
c     6
Name: a series, dtype: int64


In [65]:
## Division
s / 2

b    2.5
a    1.5
f    0.5
c    1.5
Name: a series, dtype: float64


In [66]:
## Integer division
s // 2

b    2
a    1
f    0
c    1
Name: a series, dtype: int64


In [67]:
## Modulo
s % 2

b    1
a    1
f    1
c    1
Name: a series, dtype: int64


In [68]:
## Raising to power
s ** 2

b    25
a     9
f     1
c     9
Name: a series, dtype: int64


In [69]:
## Mathematical functions from Numpy
np.exp(s)

b    148.413159
a     20.085537
f      2.718282
c     20.085537
Name: a series, dtype: float64

The same applies of course logical operators such as equality or negation.
Vectorization of logical expressions is very useful for creating masks for boolean indexing.

In [73]:
x = pd.Series(np.random.normal(0, 1, (10,)))
x

#x > 0
x[x > 0]

0   -1.435156
1    2.182012
2    1.116737
3    1.115684
4    0.420821
5   -0.332657
6    0.756088
7    0.860299
8    0.907646
9   -0.932923
dtype: float64

1    2.182012
2    1.116737
3    1.115684
4    0.420821
6    0.756088
7    0.860299
8    0.907646
dtype: float64

In [74]:
s

b    5
a    3
f    1
c    3
Name: a series, dtype: int64


In [75]:
## Equality
s == 3

b    False
a     True
f    False
c     True
Name: a series, dtype: bool


In [ ]:
## Negation of boolean expression
print(~(s == 3))
## If course equivalent to
print(s != 3)

Similarly, most of the standard Numpy mathematical functions vectorize properly over Pandas series as long as they are of numeric `dtype`.

In [76]:
## Exponentiation
np.exp(s)

In [76]:
## Natural logarithm
np.log(s)

In [76]:
## Trigonometric functions (e.g. sine)
np.sin(s)

b    148.413159
a     20.085537
f      2.718282
c     20.085537
Name: a series, dtype: float64
b    1.609438
a    1.098612
f    0.000000
c    1.098612
Name: a series, dtype: float64
b   -0.958924
a    0.141120
f    0.841471
c    0.141120
Name: a series, dtype: float64


### Series | broadcasting & alignment

As it was already mentioned, broadcasting in Pandas is very different than in Numpy as it is organized around the idea of _labels alignment_.
Luckily, this also means that the rules of broadcasting are somewhat simpler in Pandas.

Alignment of labels is based on the notion of union of sets. If we have two sets $A = \{x,y\}$ and $B = \{y,z\}$ then their union is defined as:

$$A \cup B = \{x, y, z\}$$

In other words a union is a set of all unique elements together.

In Pandas any operation between two or more series (we will extend this discussion for data frames later on) is preceeded by the axes alignment
stage in which axes of both series are transformed to the union of the two axes. We can see this in action in the chunk below.

In [77]:
s1 = pd.Series([1, 2], index=['a', 'b'])
s2 = pd.Series([3, 4], index=['a', 'c'])

print(s1)
print(s2)

s1 + s2

a    1
b    2
dtype: int64
a    3
c    4
dtype: int64


a    4.0
b    NaN
c    NaN
dtype: float64

What happened? To see what is going on step-by-step we will have to replicate the axes alignment stage ourselves. To do so we will have to
use a few methods of the index objects. For now we show them without much commentary as we will discuss index type later on.

In [81]:
# The first step is to compute index union
index_union = s1.index.union(s2.index)
index_union = s2.index.union(s1.index)
index_union

Index(['a', 'b', 'c'], dtype='object')

In [82]:
# The second step is to reindex the first series
s1 = s1.reindex(index_union)
s1

a    1.0
b    2.0
c    NaN
dtype: float64

In [83]:
# The third step is to reindex the second series
s2 = s2.reindex(index_union)
s2

a    3.0
b    NaN
c    4.0
dtype: float64

In [84]:
# The fourth and the last step is to perform the actual addition
s1 + s2

a    4.0
b    NaN
c    NaN
dtype: float64

In [86]:
x1 = pd.Series([1, 2], index=['a', 'b'])
x2 = pd.Series([10, 20], index=['b', 'a'])

print(x1)
print(x2)

x1 + x2

a    1
b    2
dtype: int64
b    10
a    20
dtype: int64


a    21
b    12
dtype: int64

We get two NaN values as we have two NaNs at different positions in two series and as we know NaN values destroy any computation they appear in.

The above example shows that NaN value can pop up frequently in Pandas as a byproduct of the axes alignment process.

Thus, it is very important to understand how they work.

## Series | NaN values

**THIS DISCUSSION GENERALIZES TO DATA FRAMES**

In general Pandas NaNs are just standard numpy `np.nan` objects, so we already know their mechanics.
However, they way they are handled in Pandas objects such as `Series` is slightly different.

Perhaps the most important difference is the fact that aggregating methods in Pandas such as `.sum()` or `.mean()` (discussed in the next section) skip NaN values by default. Moreover, standard Python `None` values are also treated as NaN in Pandas.

In [87]:
s12 = s1 + s2
s12

a    4.0
b    NaN
c    NaN
dtype: float64

In [88]:
# Discard NaNs (default behavior)
s12.sum()

4.0

In [89]:
# Do not discard NaNs
s12.sum(skipna=False)

nan

The other main difference is the thing that we already observed. NaN values are created in Pandas when new cells are created during axes alignment.

We can easily remove NaN values from a series with `.dropna` method.

In [90]:
x = pd.Series([1, 2, np.nan, 4])
print(x)
x.dropna()

0    1.0
1    2.0
2    NaN
3    4.0
dtype: float64


0    1.0
1    2.0
3    4.0
dtype: float64

In [91]:
x.isna()

0    False
1    False
2     True
3    False
dtype: bool

In [93]:
x[x.isna()]

2   NaN
dtype: float64

In [94]:
# The above is equivalent to
x[~x.isna()]
x.dropna()

0    1.0
1    2.0
3    4.0
dtype: float64

0    1.0
1    2.0
3    4.0
dtype: float64

In [96]:
# Both `np.nan` and `None` are treated as NaN in Pandas
x = pd.Series([1, np.nan, None])
x

x.isna()

In [96]:
# Alias method
x.isnull()

0    1.0
1    NaN
2    NaN
dtype: float64

0    False
1     True
2     True
dtype: bool
0    False
1     True
2     True
dtype: bool


It is also possible to fillin value in place of NaNs.

In [101]:
x = pd.Series([1, 2, np.nan, np.nan, 5], index=['a', 'b', 'c', 'd', 'e'])
print(x)

x.fillna(99)

x.fillna(x.mean())

a    1.0
b    2.0
c    NaN
d    NaN
e    5.0
dtype: float64


a     1.0
b     2.0
c    99.0
d    99.0
e     5.0
dtype: float64

a    1.000000
b    2.000000
c    2.666667
d    2.666667
e    5.000000
dtype: float64

In [103]:
x.fillna(x.mean())

x

a    1.000000
b    2.000000
c    2.666667
d    2.666667
e    5.000000
dtype: float64

a    1.0
b    2.0
c    NaN
d    NaN
e    5.0
dtype: float64

In [99]:
# The above is equivalent to
x[x.isna()] = x.mean()
x

a    1.000000
b    2.000000
c    2.666667
d    2.666667
e    5.000000
dtype: float64

In [106]:
x = pd.Series([1, 2, np.nan, np.nan, 5], index=['a', 'b', 'c', 'd', 'e'])
x
# It is possible to fill different values for different labels
x.fillna({ 'c': 3, 'd': 4})

a    1.0
b    2.0
c    NaN
d    NaN
e    5.0
dtype: float64

a    1.0
b    2.0
c    3.0
d    4.0
e    5.0
dtype: float64

In [ ]:
# The above is equivalent to
x[x.isna()] = [3, 4]
x

## Series | Aggregation

**The same methods are available also for data frames**

Pandas objects such as `Series` and `DataFrame`s support aggregation methods known from numpy. Below we list some of them.

In the case of series there is no much to aggregation as it always leads to a single number.
However, in the case of data frames we can choose axes we want to aggregato over as it we can in Numpy. We discuss this issue later on.

In [107]:
# Sum
s.sum()
# Mean
s.mean()
# Variance
s.var()
# Standard deviation
s.std()
# Minimum
s.min()
# Maximum
s.max()

12

3.0

2.6666666666666665

1.632993161855452

1

5

We can also ask for labels of maximum and minimum elements.

In [108]:
print(s)
print(s.idxmax())
print(s.idxmin())

b    5
a    3
f    1
c    3
Name: a series, dtype: int64
b
f


Moreover, we can use standard boolean aggregators.

In [109]:
print(s)

print((s == 3).all())
print((s == 3).any())

b    5
a    3
f    1
c    3
Name: a series, dtype: int64
False
True


## Series | Testing membership

A very useful feature of Pandas series is the fact that they provide easy-to-use and efficient method to test whether values
in a series are in some set of values. Below we extract from a series of positive integers only those that are prime numbers lower than $10$.

In [110]:
np.random.seed(9)

x = pd.Series(np.random.randint(1, 20, size=(30,)))
x

0      2
1      9
2     18
3      2
4      1
5     15
6     11
7     11
8     13
9     19
10     2
11     4
12    12
13     6
14     7
15    10
16    17
17    18
18     9
19    14
20     2
21    14
22    13
23    19
24     9
25     5
26     4
27     1
28    11
29    17
dtype: int64

In [111]:
primes = [2, 3, 5, 7]

x[x.isin(primes)]

0     2
3     2
10    2
14    7
20    2
25    5
dtype: int64

### Series | Exercise 1.

Change values for lowest and highest elements to -111.

In [113]:
np.random.seed(101)

s = pd.Series(np.random.randint(0, 100, (30,)))
s[:5]

0    95
1    11
2    81
3    70
4    63
dtype: int64

In [115]:
# Your solution

0    -111
1      11
2      81
3      70
4      63
5      87
6      75
7       9
8      77
9      40
10   -111
11     63
12     40
13     60
14     92
15     64
16      5
17     12
18   -111
19     40
20     49
21     83
22      8
23     29
24     59
25     34
26     44
27     72
28     19
29     10
dtype: int64

### Series | Exercise 2.

Create two series with three numeric elements and use them to create a series with six elements filled with NaNs.
When creating the 6-elements series you can use only arithmetic operations.

HINT. Remember about the rules of alignment.

In [123]:
# Your solution

a   NaN
b   NaN
c   NaN
d   NaN
e   NaN
f   NaN
dtype: float64

### Series | Iteration

The behavior of Pandas series in standard Python for-loop iteration is of course obvious. We iterate over values.

In [124]:
x = pd.Series([1, 2, 3, 4, 5])

for i in x:
    print(i)

1
2
3
4
5


However, since a series is also a kind of a `dict` we can also iterato over index values and data values in parallel
using the `.items()` method that we know from standard dictionaries.

In [126]:
x
for idx, val in x.items():
    print(idx, "=>", val)

0    1
1    2
2    3
3    4
4    5
dtype: int64

0 => 1
1 => 2
2 => 3
3 => 4
4 => 5


In general, iterating over series should be avoided as it will be almost always slow. We should use the fact that internally
series store their data in Numpy-like arrays, so we should use vectorized operations.

However, sometimes this is not possible and we may want to apply some function elementwise to a series.
To do that we can use `.map()` method definded on series objects.

It can be used to apply a function element-wise or to map elements of a series to new values based on a dictionary or other series.

In [127]:
np.random.seed(77)

x = pd.Series(np.random.choice(['a', 'b', 'c', 'd'], size=(20,)))
x

0     d
1     d
2     a
3     a
4     d
5     b
6     a
7     d
8     a
9     b
10    d
11    b
12    a
13    d
14    a
15    d
16    c
17    a
18    c
19    a
dtype: object

In [131]:
# Now we will recode the values
value_map = {
    'a': 11,
    'b': 22,
    'c': 33,
    'd': 44
}
x.map(value_map)

0     44
1     44
2     11
3     11
4     44
5     22
6     11
7     44
8     11
9     22
10    44
11    22
12    11
13    44
14    11
15    44
16    33
17    11
18    33
19    11
dtype: int64

In [132]:
# We can do the same with a function
def value_map(x):
    if x == 'a':
        return 11
    if x == 'b':
        return 22
    if x == 'c':
        return 33
    if x == 'd':
        return 44
    return np.nan

x.map(value_map)

0     44
1     44
2     11
3     11
4     44
5     22
6     11
7     44
8     11
9     22
10    44
11    22
12    11
13    44
14    11
15    44
16    33
17    11
18    33
19    11
dtype: int64

## Series | Exercise 3.

You are provided with a set of random codes composed of ASCII letters. Your task is to convert them to shorter codes according
to the following rules:

1. The shorter code should start with 1 if the longer code starts with A, with 2 if the longer starts with B, with 3 if the longer starts with C
and with 0 if the longer starts with any other letter.
2. Then append uppercase # (hash sign).
3. Then the length of the longer code should be appended.

**Example**

Long code: BACFTYSBFTYSGC

Short code: 2#14

In [133]:
from string import ascii_uppercase
long_codes = pd.Series("".join(np.random.choice(list(ascii_uppercase), size=(i,))) for i in np.random.randint(10, 50, (30,)))
long_codes

0                            CZEMDJYSOAYHXVTULCGOHGCN
1                           PLCXVYBBCPGVZUMNLYACGWTUE
2                                HPGGGLIKESQZHUXXEAJP
3                                   BTTHUZIECRSAMCZHE
4       AHRSZGUKBGRPBMREOYDLDKAYHKMCSPALYJZGVFAUCTVDW
5                                         DGKJCEBOMPG
6                               BMJIXPGJIOICAJPKHLACW
7                         OMKRQLSKUQYJBKDDTVKZKSJBZOH
8                                    TJYHSQKSYRHZZLYB
9                      PSTQDRWGIDHFNHWFBWPGYWCCTLIRIR
10    TQVBGEEMOZZWPVLGXYYSGCGYOFTEMEFGSPDCXLOCFHJHAJQ
11        GNAMVXSXVLPOPWMZDMQUAEHZBULUWTGBLMBZVTTFVFF
12                                        BBQQNBRZBCO
13                                     FRSCLJRIPZABBQ
14            UGKJISBTVFXMVODLOURUGVHCFNMSNASKCIBMYHH
15             DLBWFKTBAFXSHJPHPBTVGSWENSBYLEVWTKVJZQ
16                                         EWWTKXEDAE
17             LTKGTFJMSYYRWJCIPFOWQPVEPGRLKSVXWYROCS
18                OMLVMNAWPX

In [135]:
## Your solution

0     3#24
1     0#25
2     0#20
3     2#17
4     1#45
5     0#11
6     2#21
7     0#27
8     0#16
9     0#30
10    0#47
11    0#43
12    2#11
13    0#14
14    0#39
15    0#38
16    0#10
17    0#38
18    0#35
19    0#47
20    0#21
21    0#13
22    0#28
23    0#46
24    0#31
25    0#32
26    0#24
27    0#47
28    0#36
29    0#27
dtype: object

## Index | Explicit labeling for axes

As we already discussed index object is an integral element of every series (and as we will see later also data frame).
It provides explicit labeling for an axis along which elements of a series are aligned.
In Numpy indexes were implicit and derived just from ordering of elements. In Pandas they are full-fledged Python objects
with a lot of their functionalities, so it is important that we understand their mechanics before discussing the most
important and most complex type in Pandas, that is, `DataFrame`.

Indexes may be created with a constructor in a similar as way as series. Like series, they can also be assigned with names.

In [136]:
import pandas as pd

In [142]:
pd.Series([1,2,3], index=('a', 'b', 'c')).index

Index(['a', 'b', 'c'], dtype='object')

In [138]:
idx = pd.Index(['a', 'b', 'c'])
idx

Index(['a', 'b', 'c'], dtype='object')

In [143]:
# Named index
idx = pd.Index(['a', 'b', 'c'], name='an index')
idx

Index(['a', 'b', 'c'], dtype='object', name='an index')

Indexes can be used passed to series constructors.

In [144]:
pd.Series([1, 11, 22], index=idx)
# Note that the name of the index is printed

an index
a     1
b    11
c    22
dtype: int64

In general, indexes behave like multisets and they may contain duplicate labels. However, this situation should be usually avoided
as some procedures in Pandas are not implemented for this case and will raise an error. Moreover, this kind of ambiguity
will sooner or later lead to problems and/or errors in computations.

In [145]:
# Duplicate labels are problematic
s = pd.Series([1, 2, 3, 4], index=['a', 'b', 'a', 'c'])
s

a    1
b    2
a    3
c    4
dtype: int64

In [148]:
s['a']

a    1
a    3
dtype: int64

But they may be also useful in some cases. We will discuss this issue later.

Indexes and labels in Pandas can be very powerful, but sometimes they can be annoying and stand in our way. A typical example of situation
like that is when we have two series of the same length which we want to add together, but they come from different sources and may have
different indexes, so in the alignment stage NaN values will be produced.

Luckily, indexes can be reset and changed to generic integer indexes (similar to implicit Numpy indexes) at any moment.
We illustrate this below.

In [150]:
s1 = pd.Series([1, 2, 3])
s2 = pd.Series([4, 5, 6], index=['a', 'b', 'c'])

s1 + s2

0   NaN
1   NaN
2   NaN
a   NaN
b   NaN
c   NaN
dtype: float64

This did not work becuase labels are completely different between the series. The first one has generic integer labels, but the second
one has string labels. To solve this problem we can reset index of the second series.

In [153]:
s2
s2.reset_index(drop=True)

a    4
b    5
c    6
dtype: int64

0    4
1    5
2    6
dtype: int64

In [154]:
s1.reset_index(drop=True) + s2.reset_index(drop=True)

0    5
1    7
2    9
dtype: int64

In [155]:
s1.reset_index(drop=False)

,index,0
0,0,1
1,1,2
2,2,3


In [156]:
s2.name = 'Some Series'
s2.index.name = 'An Index'
s2
s2.reset_index(drop=False)

An Index
a    4
b    5
c    6
Name: Some Series, dtype: int64

,An Index,Some Series
0,a,4
1,b,5
2,c,6


A new index can added to a series by assigning to the `.index` attribute.

In [157]:
s = pd.Series([1, 2, 3])
s
idx = pd.Index(['a', 'b', 'c'])
s.index = idx
s

0    1
1    2
2    3
dtype: int64

a    1
b    2
c    3
dtype: int64

In [160]:
# The above is equivalent to
s = pd.Series([1, 2, 3])
s.index = ['a', 'b', 'c']
s.index

Index(['a', 'b', 'c'], dtype='object')

However, if we try to assign an index that is longer (or shorter) than our data series we will get an error.

In [164]:
s = pd.Series([1, 2, 3])
s.index = ['a', 'b', 'c', 'd']

ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In order to reshape an index of an existing series we have to use `.reindex()` method. This method can be also used to reorder labels
of a series (and its values as well). New labels are assigned with NaN values.

In [166]:
s = pd.Series([1, 2, 3, 4, 5], index=['a', 'b', 'c', 'd', 'e'])
s

a    1
b    2
c    3
d    4
e    5
dtype: int64

In [169]:
#s.reindex(['a', 'd', 'e'])
s.reindex(['a', 'f', 'b', 'd'])

a    1.0
f    NaN
b    2.0
d    4.0
dtype: float64

A very important kind of operation one can perform on indexes is to combine them in various ways.
In general indexes are treated as sets, so we can define basic set operations between them such as union and intersection.

As we already discussed, the alignment of labels (Pandas broadcasting) involves creation of a union of indexes of two series.
However, in some cases we may want think of different combinations of indexes. Below we study some possibilities.

In [172]:
s1 = pd.Series(range(10), index=range(10))
s2 = pd.Series(range(10, 20), index=range(5, 15))
s1
s2

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int64

5     10
6     11
7     12
8     13
9     14
10    15
11    16
12    17
13    18
14    19
dtype: int64

In [173]:
s1 + s2

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5     15.0
6     17.0
7     19.0
8     21.0
9     23.0
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
dtype: float64

We can use union of the indexes for instance to combine the two series in such a way that they are added together where possible
and existing values are used where one series is missing.

In [174]:
idx_union = s1.index.union(s2.index)
idx_union

RangeIndex(start=0, stop=15, step=1)

In [175]:
s1.reindex(idx_union).fillna(0) + s2.reindex(idx_union).fillna(0)

0      0.0
1      1.0
2      2.0
3      3.0
4      4.0
5     15.0
6     17.0
7     19.0
8     21.0
9     23.0
10    15.0
11    16.0
12    17.0
13    18.0
14    19.0
dtype: float64

Similarly we can use intersection of indexes to limit our results only to those cases for which we have full information.

In [176]:
idx_intersect = s1.index.intersection(s2.index)

(s1 + s2).reindex(idx_intersect)

5    15.0
6    17.0
7    19.0
8    21.0
9    23.0
dtype: float64

In [177]:
idx1 = pd.Index([1, 2, 3, 4])
idx2 = pd.Index([3, 4, 5, 6])

idx1.difference(idx2)

Int64Index([1, 2], dtype='int64')

In [180]:
idx1
idx2

idx1.difference(idx2)
idx2.difference(idx1)

Int64Index([1, 2, 3, 4], dtype='int64')

Int64Index([3, 4, 5, 6], dtype='int64')

Int64Index([1, 2], dtype='int64')

Int64Index([5, 6], dtype='int64')

In [181]:
idx1 = pd.Index([1, 2, 3, 4])
idx2 = pd.Index([3, 4, 5, 6])

idx1.symmetric_difference(idx2)

Int64Index([1, 2, 5, 6], dtype='int64')

In the same fashion we can use symmetric difference (labels in one of the index but not in both).

In [182]:
idx_symmetric = s1.index.symmetric_difference(s2.index)

In [183]:
s1.reindex(idx_symmetric)

0     0.0
1     1.0
2     2.0
3     3.0
4     4.0
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
dtype: float64

In [184]:
s2.reindex(idx_symmetric)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
10    15.0
11    16.0
12    17.0
13    18.0
14    19.0
dtype: float64

In [187]:
s1 + s2
s1.reindex(idx_symmetric).fillna(0) + s2.reindex(idx_symmetric).fillna(0)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5     15.0
6     17.0
7     19.0
8     21.0
9     23.0
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
dtype: float64

0      0.0
1      1.0
2      2.0
3      3.0
4      4.0
10    15.0
11    16.0
12    17.0
13    18.0
14    19.0
dtype: float64

### Index | Exercise 1.

You are provided with two data series with measurements for subjects identified with integer indexes. The first series corresponds to the first
trial and second one to the second trial. However, some subjects participated only in one of the trials.

You have to execute the following tasks:

1. Compute a series with sums of scores for all subjects.
2. Compute a series with average scores for all subjects.
3. Find ids (labels) for subjects with highest and lowest average score.
4. Compute average score among subjects who participated in both trials.
5. Compute average score among subjects who participated in only one of the trials.

In [211]:
import numpy as np
np.random.seed(101)

ids = np.arange(25)
trial1 = pd.Series(np.random.normal(100, 15, (22,)), index=np.random.choice(ids, size=(22,), replace=False))
trial2 = pd.Series(np.random.normal(115, 30, (13,)), index=np.random.choice(ids, size=(13,), replace=False))

In [ ]:
# Your solution